# PyTorch --> PyTorch Lightning

## Image Classification with PyTorch

In [ ]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 4

In [ ]:
train_data = torchvision.datasets.CIFAR10('/Users/aniket/datasets/',
                                          download=True,
                                          transform=transform)

val_data = torchvision.datasets.CIFAR10('/Users/aniket/datasets/',
                                        train=False,
                                        download=True,
                                        transform=transform)

trainloader = torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=True)

valloader = torch.utils.data.DataLoader(val_data,
                                        batch_size=batch_size,
                                        shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse',
           'ship', 'truck')

In [ ]:
# functions to show an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

## Load Model

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

## Define Loss & Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
NUM_EPOCHS = 2

## Training Step

In [ ]:
# Calculate loss
def train_step(model, data):
    inputs, labels = data

    y_pred = net(inputs)
    loss = criterion(y_pred, labels)
    return loss

# Training Loop 

In [ ]:
for epoch in range(NUM_EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):

        # zero the parameter gradients
        optimizer.zero_grad()

        loss = train_step(net, data)

        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

# ⚡️ PyTorch Lightning 

## Lightning Design Philosophy

![Design Phil](https://camo.githubusercontent.com/a3dda9b3053f7093ac1ccc30b41c0f367b171010770252528c30551a2407cad5/68747470733a2f2f706c2d626f6c74732d646f632d696d616765732e73332e75732d656173742d322e616d617a6f6e6177732e636f6d2f7068696c6f736f70686965732e6a7067)

In [ ]:
import pytorch_lightning as pl

In [ ]:
class LitModel(pl.LightningModule):

    def __init__(self, lr):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        y_pred = self(inputs)
        loss = criterion(y_pred, labels)
        
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        y_pred = self(inputs)
        loss = criterion(y_pred, labels)
        
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams['lr'])
        return optimizer

In [ ]:
class CIFARDataModule(pl.LightningDataModule):

    def __init__(self, batch_size: int = 64):
        super().__init__()
        self.batch_size = batch_size
        
        self.train_data = torchvision.datasets.CIFAR10(
            '/Users/aniket/datasets/', download=True, transform=transform)

        self.val_data = torchvision.datasets.CIFAR10('/Users/aniket/datasets/',
                                                     train=False,
                                                     download=True,
                                                     transform=transform)
        

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_data,
                                           batch_size=self.batch_size,
                                           shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_data,
                                           batch_size=self.batch_size,
                                           shuffle=False)
    
cifar_dm = CIFARDataModule(batch_size=64)

# Logger & Callbacks

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger

tb_logger = TensorBoardLogger("tb_logs", name="my_model")
csv_logger = CSVLogger('./')

In [ ]:
lit_model = LitModel(1e-3)
trainer = pl.Trainer(logger=[tb_logger, csv_logger])

In [ ]:
trainer.fit(lit_model, datamodule=cifar_dm)

#### References:
1. [PyTorch: TRAINING A CLASSIFIER](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)
